# Import libraries

In [17]:
import numpy as np
import cv2
import os

# 1 Membuat Training Data

In [18]:
# Baca LBP Cascade
lbpcascade_frontalface = 'lbpcascades/lbpcascade_frontalface.xml'

In [19]:
# fungsi deteksi wajah pada opencv
def detek_wajah(input_gbr):
    # konversi gambar ke kabuan
    gbr = cv2.cvtColor(input_gbr, cv2.COLOR_BGR2GRAY)
    
    # klasifikasi cascade
    cascade_wajah = cv2.CascadeClassifier(lbpcascade_frontalface)
    
    # deteksi dengan multiscale 
    wajah = cascade_wajah.detectMultiScale(gbr, scaleFactor=1.1, minNeighbors=4);
    
    #jika tidak ada wajah terdeteksi kembali ke gambar yang asli
    if (len(wajah) == 0):
        return None, None
    
    # ekstrak area wajah
    (x, y, w, h) = wajah[0]
    # kembalikan hanya bagian wajah dari gambar
    return gbr[y:y+w, x:x+h], wajah[0]

## Membuat Persegi untuk Input Deteksi Wajah

In [20]:
def gbr_persegi(gbr, persegi):
    # gambar persegi pada wajah yang terdeteksi
    (x, y, w, h) = persegi
    cv2.rectangle(gbr, (x, y), (x+w, y+h), (0, 255, 0), 2)

def gbr_text(gbr, text, x, y):
    # tampilkana tek pada wajah yang terdeteksi
        cv2.putText(gbr, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

# 2. Membuat Training Data

## Merekam Wajah Untuk Training Data

In [23]:
# buka webcam dan baca setiap frame. Setiap frame melewati fungsi deteksi wajah dan
# wajah yang terdeteksi disimpan dalam folder yang akan digunakan untuk pelatihan

video = cv2.VideoCapture(0)
jumlah = 0
while True:
    ret, frame = video.read()
    wajah_pilih, persegi = detek_wajah(frame)
    if wajah_pilih is not None:
        jumlah +=1
        gbr_persegi(frame, persegi)
        # change the folder path as per requirement
        cv2.imwrite("rekam_wajah/0/"+str(jumlah)+ ".jpg", wajah_pilih)
    cv2.imshow("frame", frame)
    # Escape untuk keluar
    if cv2.waitKey(1)==27:
        break
video.release()
cv2.destroyAllWindows() 

In [29]:
# Baca gambar dari folder dan siapkan set pelatihan dan label masing-masing
deteksi_wajah = []
label_wajah = []

def prepare_training_data(path_training_data):
    
    # baca gambar dari folder dan siapkan set data pelatihan
    #  mengembalikan daftar wajah dan label yang terdeteksi
    
    gambar_training = os.listdir(path_training_data)
    for nama_di_direktori in gambar_training:
        label = int(nama_di_direktori)
        path_training_gbr = path_training_data + "/" + nama_di_direktori
        nama_training_gbr = os.listdir(path_training_gbr)
        for image_name in nama_training_gbr:
            gbr_path = path_training_gbr  + "/" + image_name
            gambar = cv2.imread(gbr_path, 0)
            deteksi_wajah.append(gambar)
            label_wajah.append(label)

    return deteksi_wajah, label_wajah

In [30]:
deteksi_wajah, label_wajah = prepare_training_data('rekam_wajah')

In [31]:
# print total label dan wajah yang di training
print("Total wajah: ", len(deteksi_wajah))
print("Total label: ", len(label_wajah))

Total wajah:  29
Total label:  29


In [32]:
# buat pengenal wajah LBPH
lbphfaces_recognizer = cv2.face. LBPHFaceRecognizer_create(radius=2, neighbors=8)
# latih pengenalan wajah dari wajah latihan 
lbphfaces_recognizer.train(deteksi_wajah, np.array(label_wajah))

# 3. Jalankan Pengenalan Wajah

In [34]:
# buka webcam, baca setiap frame dan berikan ke model terlatih. 
# Untuk prediksi biner: Jika nilai keyakinan prediksi kurang dari nilai ambang batas 
# maka atur teks label sama dengan nama pengguna  selain itu tidak diketahui. 
# Untuk prediksi multikelas: atur label_text ke label

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    wajah_pilih, persegi = detek_wajah(frame)
    
    if wajah_pilih is not None:
        label= lbphfaces_recognizer.predict(wajah_pilih)
        tags = ['Saya', 'Nanda']
        if label[1]<75:
            
            label_text = tags[label[0]]
        else: 
            label_text = 'Tidak dikenal'
        gbr_persegi(frame, persegi)
        if label[1]<75:
            gbr_text(frame, label_text, persegi[0], persegi[1]-5)
        else:
            gbr_text(frame, label_text, persegi[0], persegi[1]-5)
    cv2.imshow("frame", frame)
    #escape untuk keluar
    if cv2.waitKey(1)==27:
        break
video.release()
cv2.destroyAllWindows()